<img src="https://raw.githubusercontent.com/fluidml/fluidml/main/logo/fluid_ml_logo.png" width=400 height=400 />

# Transformer based Sequence to Sequence Translation using FluidML
In this notebook, we utilize FluidML to implement a complete ML pipeline that performs text translation from German to English.  
Our translation pipeline consists of the following tasks:
- **Dataset loading**: Downloads and parses the Multi30K dataset used for translation.
- **Tokenizer training**: Trains and saves a Byte Pair Encoding (BPE) Tokenizer used for text encoding and decoding.
- **Dataset encoding**: Encodes the dataset with the trained BPE Tokenizer.
- **Model training**: Trains the Transformer based Sequence to Sequence Model.
- **Model selection**: Selects from all trained model variations (different hyperparameter sweeps) the best performing one based on the validation set.
- **Model evaluation**: Evaluates the best performing model on the test set by calculating the test loss and the bleu score.

With FluidML, all of these steps are naturally implemented as individual tasks which register their dependencies and are chained together to a task graph. This graph is then executed in parallel by FluidML and all results are stored persistently in a local file store (see the Storage section for details).

## Setup

To run this example it makes sense to install FluidML with the additional example requirements (Of course you can also manually install all dependencies. Check `transformer_seq2seq_translation.py` for a complete list). `cd` into the cloned fluidml directory and run:
```bash 
pip install .[examples,rich-logging]
```

**Note**: Due to the limitation of multiprocessing and jupyter, we have to import our defined tasks and some helper classes from a separate script. Hence, our task definitions are located in `transformer_seq2seq_translation.py`, which not only implements the tasks but also the entire functionality of this example. So the interested reader can also go ahead and execute the just mentioned script. In order to still make this notebook self-explanatory, we provide Markdown code snippets of the individual task implementations at the place where we would have defined the task.

In [ ]:
# Python internal imports
import multiprocessing
import os
from typing import List, Dict, Any, Optional, Tuple

# External imports
import torch
from tokenizers import Tokenizer

# FluidML imports
from fluidml import Flow, Swarm
from fluidml.common import Task, Resource
from fluidml.flow import GridTaskSpec, TaskSpec

# Task imports, file store import and resource class import (see above note)
from transformer_seq2seq_translation import DatasetLoading, TokenizerTraining, DatasetEncoding, Training, ModelSelection, Evaluation
from transformer_seq2seq_translation import TaskResource, MyLocalFileStore

**Note 2**: If you want to use FluidML's logging capability, please configure a logger using Python's `logging` API. For convenience, we provide a simple utility function which configures a visually appealing logger (using a specific handler from the `rich` library).

In [ ]:
from fluidml.common.logging import configure_logging
configure_logging()

## Storage - Saving Objects with FluidML

Before we start with the implementation of our translation pipeline, we take a brief look at FluidML's storage API.  
Out of the box FluidML provides three different storage options, which share the same interface (the interested user can implement his own storage, as long as the storage class inherits from `fluidml.storage.base.ResultsStore` and implements all abstract methods):
- **InMemoryStore**: If no store object is provided, this is internally the default. Every saved object is stored in an in-memory manager dictionary, which is shared across all tasks and processes. Once the entire pipeline is executed, the results dictionary is returned to the user and it is the user's responsibility to actually save the results e.g. to disc. This store is only recommended for quick prototyping and small result objects, since intermediate task results cannot be stored persistently and the memory might not be sufficient to hold all task results.
- **LocalFileStore**: A persistent file store implementation that out of the box supports saving files as .json and .pickle. It can be easily extended by the user to support arbitrary file types and save options.
- **MongoDBStore**: A persistent MongoDBStore implementation, which stores saved objects as binary strings via GridFS in a Mongo DB.

In this example we utilize the `LocalFileStore` and extend it with our own custom saving types.  
In order to save an object within a task, one simply calls
```python
self.save(obj=model_state_dict, name='best_model', type_='torch')
self.save(obj=some_dict, name='some_dict', type_='json')
self.save(obj=some_serializable_obj, name='some_serializable_obj', type_='pickle')
```
Below, we implement `MyLocalFileStore`, which inherits from `LocalFileStore` and extends it by adding save and load functions for torch models and tokenizer objects.

```python
class MyLocalFileStore(LocalFileStore):
    def __init__(self, base_dir: str):
        super().__init__(base_dir=base_dir)

        self._save_load_fn_from_type['torch'] = (self._save_torch, self._load_torch)
        self._save_load_fn_from_type['tokenizer'] = (self._save_tokenizer, self._load_tokenizer)

    @staticmethod
    def _save_torch(name: str, obj: Any, run_dir: str):
        torch.save(obj, f=os.path.join(run_dir, f'{name}.pt'))

    @staticmethod
    def _load_torch(name: str, run_dir: str) -> Any:
        return torch.load(os.path.join(run_dir, f'{name}.pt'))

    @staticmethod
    def _save_tokenizer(name: str, obj: Tokenizer, run_dir: str):
        obj.save(os.path.join(run_dir, f'{name}.json'))

    @staticmethod
    def _load_tokenizer(name: str, run_dir: str) -> Tokenizer:
        return Tokenizer.from_file(os.path.join(run_dir, f'{name}.json'))
```

## Task Definitions

The following 6 sections describe our task definitions in detail.

### 1. Dataset Loading

For this example we use the [Multi30K](https://github.com/multi30k/dataset) translation dataset, considering only German and English. The dataset was published with a fixed split of 29,000 train, 1,000 validation and 1,000 test German-English text pairs.

We implement this Task by creating a custom `DatasetLoading` class, which inherits from FluidML's `Task` class. To comply with our interface a custom task just has to implement a `run()` method, which FluidML will execute internally.

Here is our complete implementation of DatasetLoading (imported above):


```python
import gzip
import requests


class DatasetLoading(Task):
    def __init__(self,
                 base_url: str,
                 data_split_names: Dict[str, List]):
        super().__init__()
        self.base_url = base_url
        self.data_split_names = data_split_names

    @staticmethod
    def download_and_extract_gz_from_url(url: str) -> List[str]:
        # download gz compressed data
        data_gz = requests.get(url=url)
        # decompress downloaded gz data to bytes object
        data_bytes = gzip.decompress(data_gz.content)
        # decode bytes object to utf-8 encoded str and convert to list by splitting on new line chars
        data = data_bytes.decode('utf-8').splitlines()
        return data

    def run(self):
        for split_name, files in self.data_split_names.items():
            dataset = {}
            for file_name in files:
                url = self.base_url + file_name
                language = file_name.split('.')[1]
                data = DatasetLoading.download_and_extract_gz_from_url(url=url)
                dataset[language] = data

            self.save(obj=dataset, name=f'{split_name}_data', type_='json')

```

### 2. Tokenizer Training

We utilize Huggingface's [Tokenizers](https://github.com/huggingface/tokenizers) library to train a Byte Pair Encoding (BPE) tokenizer for the German and for the English sentences.


```python
from tokenizers.implementations import CharBPETokenizer
from tokenizers.processors import TemplateProcessing


class TokenizerTraining(Task):
    def __init__(self, vocab_size: int, min_frequency: int):
        super().__init__()
        
        self.vocab_size = vocab_size
        self.min_frequency = min_frequency

    def train_tokenizer(self, data: List[str]):
        # initialize and train a tokenizer
        tokenizer = CharBPETokenizer()
        tokenizer.train_from_iterator(iterator=data,
                                      vocab_size=self.vocab_size,
                                      min_frequency=self.min_frequency,
                                      special_tokens=['<unk>', '<bos>', '<eos>', '<pad>'],
                                      show_progress=True)
        
        # add template rule to automatically add <bos> and <eos> to the encoding
        tokenizer.post_processor = TemplateProcessing(
            single="<bos> $A <eos>",
            pair=None,
            special_tokens=[
                ("<bos>", tokenizer.token_to_id("<bos>")),
                ("<eos>", tokenizer.token_to_id("<eos>")),
            ],
        )
        return tokenizer

    def run(self, train_data: Dict[str, List[str]]):
        # train german tokenizer
        de_tokenizer = self.train_tokenizer(data=train_data['de'])

        # train english tokenizer
        en_tokenizer = self.train_tokenizer(data=train_data['en'])

        # save tokenizers
        self.save(obj=de_tokenizer, name='de_tokenizer', type_='tokenizer')
        self.save(obj=en_tokenizer, name='en_tokenizer', type_='tokenizer')

```

### 3. Dataset Encoding

We use the trained tokenizers for German and English to encode the previously saved datasets and save them as json files.  
**Note**: FluidML automatically collects the required task inputs from the saved predecessor task results; in this case the datasets saved from `DatasetLoading` and the tokenizers trained in `TokenizerTraining`.


```python
class DatasetEncoding(Task):
    def __init__(self):
        super().__init__()

    @staticmethod
    def encode_data(data: Dict[str, List[str]],
                    src_tokenizer: Tokenizer,
                    trg_tokenizer: Tokenizer) -> List[Tuple[List[int], List[int]]]:

        src_encoded = src_tokenizer.encode_batch(data['de'])
        trg_encoded = trg_tokenizer.encode_batch(data['en'])
        return [(src.ids, trg.ids) for src, trg in zip(src_encoded, trg_encoded)]

    def run(self,
            train_data: Dict[str, List[str]],
            valid_data: Dict[str, List[str]],
            test_data: Dict[str, List[str]],
            de_tokenizer: Tokenizer,
            en_tokenizer: Tokenizer):

        train_encoded = DatasetEncoding.encode_data(train_data, de_tokenizer, en_tokenizer)
        valid_encoded = DatasetEncoding.encode_data(valid_data, de_tokenizer, en_tokenizer)
        test_encoded = DatasetEncoding.encode_data(test_data, de_tokenizer, en_tokenizer)

        self.save(obj=train_encoded, name='train_encoded', type_='json')
        self.save(obj=valid_encoded, name='valid_encoded', type_='json')
        self.save(obj=test_encoded, name='test_encoded', type_='json')

```

### 4. Model Training

Our dataset is encoded using the trained BPE tokenizers, so the next step is to train the actual translation model. We utilize the well known transformer architecture first described in the [Attention is all you need](https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf) paper. Both, the encoder and decoder consist of several multi-head attention layers, which are the backbone of this architecture. Since this tutorial is about FluidML, we won't go into the implementational details of a transformer model. We refer the interested reader to [Ben Trevett's tutorial on transformers for translation](https://github.com/bentrevett/pytorch-seq2seq/blob/master/6%20-%20Attention%20is%20All%20You%20Need.ipynb). Also, the pytorch transformer implementation used in this example is taken from Ben's tutorial.

**Note**: The custom pytorch dataset, `TranslationDataset`, and the batch collate callable, `BatchCollator`, implementations can be also found in the above mentioned script, `transformer_seq2seq_translation.py`, from where the task classes are imported. Due to its complexity the transformer model implementation is also imported in this notebook and can be found in the `transformer_model.py` script. The definition of the `set_seed()` function is also located in `transformer_seq2seq_translation.py`.

You also might note that we use `self.resource.seed` and `self.resource.device` in our training task without explicitly defining these in the `__init__()` method. When initializing FluidML's main classes `Flow` and `Swarm`, the user can optionally provide a list of resources that will be made available to all tasks and processes. In a machine learning context such resources could be but are not limited to seeds or cuda devices (see this example). Below we will go through how to define and provide the list of resources to `Flow` and how the resources will be automatically distributed to tasks.


```python
from datetime import datetime

import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


class Training(Task):
    def __init__(self,
                 hid_dim: int,
                 enc_layers: int,
                 dec_layers: int,
                 enc_heads: int,
                 dec_heads: int,
                 enc_pf_dim: int,
                 dec_pf_dim: int,
                 enc_dropout: float,
                 dec_dropout: float,
                 learning_rate: float,
                 clip_grad: float,
                 train_batch_size: int,
                 valid_batch_size: int,
                 num_epochs: int):
        super().__init__()

        # transformer model parameters
        self.hid_dim = hid_dim
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.enc_heads = enc_heads
        self.dec_heads = dec_heads
        self.enc_pf_dim = enc_pf_dim
        self.dec_pf_dim = dec_pf_dim
        self.enc_dropout = enc_dropout
        self.dec_dropout = dec_dropout

        # optimizer parameters
        self.learning_rate = learning_rate
        self.clip_grad = clip_grad

        # dataloader and training loop parameters
        self.train_batch_size = train_batch_size
        self.valid_batch_size = valid_batch_size
        self.num_epochs = num_epochs

    def _init_training(self, input_dim: int, output_dim: int, src_pad_idx: int, trg_pad_idx: int):
        """ Initialize all training components.
        """

        # initialize the encoder and decoder block
        enc = Encoder(input_dim,
                      self.hid_dim,
                      self.enc_layers,
                      self.enc_heads,
                      self.enc_pf_dim,
                      self.enc_dropout,
                      self.resource.device)

        dec = Decoder(output_dim,
                      self.hid_dim,
                      self.dec_layers,
                      self.dec_heads,
                      self.dec_pf_dim,
                      self.dec_dropout,
                      self.resource.device)

        # initialize the full transformer sequence to sequence model
        model = Seq2SeqTransformer(enc, dec, src_pad_idx, trg_pad_idx, self.resource.device).to(self.resource.device)

        # initialize the optimizer
        optimizer = torch.optim.Adam(model.parameters(), lr=self.learning_rate)

        # initialize the loss criterion
        criterion = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)
        return model, optimizer, criterion

    def _train_epoch(self, model, iterator, optimizer, criterion):
        """ Train loop to iterate over batches
        """
        model.train()

        epoch_loss = 0

        for i, (src, trg) in enumerate(iterator):

            optimizer.zero_grad()

            output, _ = model(src, trg[:, :-1])
            output_dim = output.shape[-1]
            output = output.contiguous().view(-1, output_dim)  # [batch size * trg len - 1, output dim]
            trg = trg[:, 1:].contiguous().view(-1)             # [batch size * trg len - 1]

            loss = criterion(output, trg)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), self.clip_grad)

            optimizer.step()
            epoch_loss += loss.item()
        return epoch_loss / len(iterator)

    @staticmethod
    def validate_epoch(model, iterator, criterion):
        """ Validation loop to iterate over batches
        """
        model.eval()

        epoch_loss = 0

        with torch.no_grad():
            for src, trg in iterator:

                output, _ = model(src, trg[:, :-1])
                output_dim = output.shape[-1]
                output = output.contiguous().view(-1, output_dim)  # [batch size * trg len - 1, output dim]
                trg = trg[:, 1:].contiguous().view(-1)             # [batch size * trg len - 1]

                loss = criterion(output, trg)
                epoch_loss += loss.item()
        return epoch_loss / len(iterator)

    def _train(self, model, train_iterator, valid_iterator, optimizer, criterion):
        """ Train loop.
        """

        best_valid_loss = float('inf')
        best_model = None

        for epoch in tqdm(range(self.num_epochs)):

            start_time = datetime.now()
            train_loss = self._train_epoch(model, train_iterator, optimizer, criterion)
            valid_loss = Training.validate_epoch(model, valid_iterator, criterion)
            end_time = datetime.now()

            # if the current validation loss is below the previous best, update the best loss and
            # save the new best model.
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                best_model = model.state_dict()
                self.save(obj=model.state_dict(), name='best_model', type_='torch')
                self.save(obj={'epoch': epoch,
                               'valid_loss': best_valid_loss}, name='best_model_metric', type_='json')

            print(f'Epoch: {epoch + 1:02} | Time: {end_time - start_time}')
            print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
            print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
        assert best_model is not None
        return best_model, best_valid_loss

    def run(self,
            train_encoded: List[Tuple[List[int], List[int]]],
            valid_encoded: List[Tuple[List[int], List[int]]],
            de_tokenizer: Tokenizer,
            en_tokenizer: Tokenizer):
        set_seed(self.resource.seed)

        # instantiate the collate fn for the dataloader
        batch_collator = BatchCollator(de_pad_idx=de_tokenizer.token_to_id('<pad>'),
                                       en_pad_idx=en_tokenizer.token_to_id('<pad>'),
                                       device=self.resource.device)

        # instantiate train and validation datasets using a pytorch's Dataset class
        train_dataset = TranslationDataset(data=train_encoded)
        valid_dataset = TranslationDataset(data=valid_encoded)

        # instantiate train and validation dataloader
        train_iterator = DataLoader(train_dataset, batch_size=self.train_batch_size, shuffle=True,
                                    collate_fn=batch_collator)
        valid_iterator = DataLoader(valid_dataset, batch_size=self.valid_batch_size, shuffle=False,
                                    collate_fn=batch_collator)

        input_dim = de_tokenizer.get_vocab_size()
        output_dim = en_tokenizer.get_vocab_size()
        src_pad_idx = de_tokenizer.token_to_id('<pad>')
        trg_pad_idx = en_tokenizer.token_to_id('<pad>')

        # instantiate all training components
        model, optimizer, criterion = self._init_training(input_dim=input_dim,
                                                          output_dim=output_dim,
                                                          src_pad_idx=src_pad_idx,
                                                          trg_pad_idx=trg_pad_idx)

        # train the model on the training set and evaluate after every epoch on the validation set
        self._train(model=model,
                    train_iterator=train_iterator,
                    valid_iterator=valid_iterator,
                    optimizer=optimizer,
                    criterion=criterion)
```

### 5. Model Selection

We have not yet talked about grid search and training several models with different hyperparameter combinations in parallel. FluidML provides a simple interface to allow just that, which we will describe further below when instantiating our tasks. For now let's assume that the previous tasks might have been executed multiple times with a set of different parameter combinations yielding several trained model variations.

This task is a so called `reduce=True` task (will be used when instantiating the task), which means that it collects the results from all predecessor variations in order to compare and select the best performing variation. In this example it selects the best modelvariation based on the validation loss performance.


```python
class ModelSelection(Task):
    def __init__(self):
        super().__init__()

    @staticmethod
    def _select_best_model_from_sweeps(training_results: List[Dict]) -> Dict:
        config = None
        best_valid_loss = float('inf')
        for sweep in training_results:
            if sweep['result']['best_model_metric']['valid_loss'] <= best_valid_loss:
                best_valid_loss = sweep['result']['best_model_metric']['valid_loss']
                config = sweep['config']
        return config

    def run(self, reduced_results: List[Dict]):
        
        # select the best run config by comparing model performances from different parameter sweeps 
        # on the validation set
        best_run_config = self._select_best_model_from_sweeps(training_results=reduced_results)

        self.save(obj=best_run_config, name='best_run_config', type_='json')
```

### 6. Model Evaluation

This is the final task in our pipeline. It expects the previously determined `best_run_config` dictionary as input, loads the corresponding best model and tokenizers and evaluates said model on the test dataset.  
First, we calculate the test set loss and perplexity. Second, we calculate the test set bleu score, since this is the standard metric of evaluating machine translation models.



```python
class Evaluation(Task):
    def __init__(self, test_batch_size: int):
        super().__init__()

        self.batch_size = test_batch_size

    def _init_model(self, train_config: Dict, input_dim: int, output_dim: int,
                    src_pad_idx: int, trg_pad_idx: int) -> nn.Module:
        """ Initialize the model and its components.
        """
        
        enc = Encoder(input_dim,
                      train_config['hid_dim'],
                      train_config['enc_layers'],
                      train_config['enc_heads'],
                      train_config['enc_pf_dim'],
                      train_config['enc_dropout'],
                      self.resource.device)

        dec = Decoder(output_dim,
                      train_config['hid_dim'],
                      train_config['dec_layers'],
                      train_config['dec_heads'],
                      train_config['dec_pf_dim'],
                      train_config['dec_dropout'],
                      self.resource.device)

        model = Seq2SeqTransformer(enc, dec, src_pad_idx, trg_pad_idx, self.resource.device).to(self.resource.device)
        return model

    def translate_sentence(self, src_encoded, bos_idx, eos_idx, model, max_len=50):
        """ Translate an encoded sentence.
        """
        
        model.eval()

        src_tensor = torch.LongTensor(src_encoded).unsqueeze(0).to(self.resource.device)
        src_mask = model.make_src_mask(src_tensor)

        with torch.no_grad():
            enc_src = model.encoder(src_tensor, src_mask)

        trg_indices = [bos_idx]
        for i in range(max_len):
            trg_tensor = torch.LongTensor(trg_indices).unsqueeze(0).to(self.resource.device)
            trg_mask = model.make_trg_mask(trg_tensor)
            with torch.no_grad():
                output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
            pred_token = output.argmax(2)[:, -1].item()
            trg_indices.append(pred_token)
            if pred_token == eos_idx:
                break

        return trg_indices[1:]

    def calculate_bleu(self, data_encoded, en_tokenizer, model, max_len=50):
        """ Calculate the bleu score on the test set.
        """
        
        trgs = []
        pred_trgs = []
        bos_idx = en_tokenizer.token_to_id('<bos>')
        eos_idx = en_tokenizer.token_to_id('<eos>')

        for src, trg in tqdm(data_encoded):

            pred_trg = self.translate_sentence(src, bos_idx, eos_idx, model, max_len)

            # cut off <eos> token
            pred_trg = pred_trg[:-1]

            pred_trg_decoded = en_tokenizer.decode(pred_trg)
            pred_trgs.append(pred_trg_decoded.split())

            trg_decoded = en_tokenizer.decode(trg)
            trgs.append(trg_decoded.split())

        return bleu_score(pred_trgs, trgs)

    def run(self, best_run_config: Dict):
        set_seed(self.resource.seed)

        # load the best model, test-data and the tokenizers based on the previously selected best run config
        model_state_dict = self.load(name='best_model', task_name='Training', task_unique_config=best_run_config)
        test_encoded = self.load(name='test_encoded', task_name='DatasetEncoding', task_unique_config=best_run_config)
        de_tokenizer = self.load(name='de_tokenizer', task_name='TokenizerTraining', task_unique_config=best_run_config)
        en_tokenizer = self.load(name='en_tokenizer', task_name='TokenizerTraining', task_unique_config=best_run_config)

        # instantiate the batch collator
        batch_collator = BatchCollator(de_pad_idx=de_tokenizer.token_to_id('<pad>'),
                                       en_pad_idx=en_tokenizer.token_to_id('<pad>'),
                                       device=self.resource.device)
        
        # instantiate the test dataset
        test_dataset = TranslationDataset(data=test_encoded)

        # instantiate the test dataloader
        test_iterator = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False, collate_fn=batch_collator)

        input_dim = de_tokenizer.get_vocab_size()
        output_dim = en_tokenizer.get_vocab_size()
        src_pad_idx = de_tokenizer.token_to_id('<pad>')
        trg_pad_idx = en_tokenizer.token_to_id('<pad>')

        # instantiate the transformer model
        model = self._init_model(train_config=best_run_config['Training'],
                                 input_dim=input_dim,
                                 output_dim=output_dim,
                                 src_pad_idx=src_pad_idx,
                                 trg_pad_idx=src_pad_idx)
        model.load_state_dict(model_state_dict)

        # instantiate the loss criterion
        criterion = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)

        # evaluate the model on the test set -> calculate the test set loss and perplexity
        test_loss = Training.validate_epoch(model=model, iterator=test_iterator, criterion=criterion)
        print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

        # calculate the model's bleu score on the test set
        bleu = self.calculate_bleu(test_encoded, en_tokenizer, model)
        print(f'BLEU score = {bleu * 100:.2f}')
```

## Create and Run the Pipeline/Task-Graph via FluidML

So far, we have looked into implementing our individual pipeline steps using FluidML's Task class and it was very straightforward.
You might be wondering, how to put these tasks together and make them work together as a single pipeline?

Thanks to FluidML's TaskSpec API, you can connect these tasks like Lego blocks :)

### 1. Instantiate Task Specs
`TaskSpec` and `GridTaskSpec` are simple wrapper classes that allow to specify task details and task arguments which will be used during instantiation of the task.
Let's go ahead and create specs for all our tasks.  

**Note 1**: If a task expects specific input objects and publishes result objects, that are relevent to successor tasks, these object names have to be registered through the `expects` and `publishes` attributes. This is necessary because of two reasons. First, FluidML internally uses this information to automatically collect all required input objects from the result store (e.g. `LocalFileStore`) and provide them to the task through the `run(*collected_inputs)` method. Second, based on this information FluidML decides whether a task still needs to be executed or whether it has been executed already in a previous run and all results can be loaded from the store.

**Note 2**: A task where no hyperparameter tuning or grid search is necessary is wrapped in the `TaskSpec` class. Likewise, tasks, where we want to perform hyperparameter tuning, e.g. `TokenizerTraining` and `Training`, are wrapped in the dedicated `GridTaskSpec` class.

Below we define for each task the necessary parameter dictionary which we feed into the task spec class.  

**Note 3**: All parameters of a `GridTaskSpec` task that are stored in a list, will be automatically expanded by `Flow` to create different task instances for each explicit parameter combination. For example, considering the `Training` task, internally flow will instantiate 4 train tasks with the cross product combinations of different `train_batch_size` and `learning_rate`.

**Note 4**: If you have to provide a parameter to a `GridTaskSpec` task, which is of type `List` so it should not get expanded, you have to wrap it again in a second list. E.g. `layer_dimensions: [[64, 128, 64]]`.

In [ ]:
# create all task specs

dataset_loading_params = {'base_url': 'https://raw.githubusercontent.com/multi30k/dataset/'
                                      'master/data/task1/raw/',
                          'data_split_names': {'train': ['train.de.gz', 'train.en.gz'],
                                               'valid': ['val.de.gz', 'val.en.gz'],
                                               'test': ['test_2016_flickr.de.gz', 'test_2016_flickr.en.gz']}}
dataset_loading_task = TaskSpec(task=DatasetLoading, task_kwargs=dataset_loading_params,
                                publishes=['train_data', 'valid_data', 'test_data'])


tokenizer_training_params = {'vocab_size': 30000,
                             'min_frequency': 2}
tokenizer_training_task = GridTaskSpec(task=TokenizerTraining, gs_config=tokenizer_training_params,
                                       expects=['train_data'],
                                       publishes=['de_tokenizer', 'en_tokenizer'])


dataset_encoding_task = TaskSpec(task=DatasetEncoding,
                                 expects=['train_data', 'valid_data', 'test_data', 'de_tokenizer', 'en_tokenizer'],
                                 publishes=['train_encoded', 'valid_encoded', 'test_encoded'])


training_params = {'hid_dim': 256,
                   'enc_layers': 3,
                   'dec_layers': 3,
                   'enc_heads': 8,
                   'dec_heads': 8,
                   'enc_pf_dim': 512,
                   'dec_pf_dim': 512,
                   'enc_dropout': 0.1,
                   'dec_dropout': 0.1,
                   'learning_rate': [0.0005, 0.001],
                   'clip_grad': 1.,
                   'train_batch_size': [64, 128],
                   'valid_batch_size': 64,
                   'num_epochs': 10}
train_task = GridTaskSpec(task=Training, gs_config=training_params,
                          expects=['train_encoded', 'valid_encoded', 'de_tokenizer', 'en_tokenizer'],
                          publishes=['best_model', 'best_model_metric'])


model_selection_task = TaskSpec(task=ModelSelection, reduce=True, expects=['best_model_metric'],
                                publishes=['best_run_config'])


evaluation_params = {'test_batch_size': 128}
evaluate_task = TaskSpec(task=Evaluation, reduce=False, task_kwargs=evaluation_params,
                         expects=['best_run_config'], publishes=[])

### 2. Registering all Task Dependencies
After having instantiated all task specs, we utilize the `requires()` method to register dependencies between tasks.

Using these task dependencies, FluidML's `Flow` class properly expands all `GridTaskSpecs` and creates a task graph. Next, FluidML's `Swarm` class schedules and performs the parallel task executions considering the registered dependencies.

In [ ]:
# register dependencies between tasks

tokenizer_training_task.requires([dataset_loading_task])
dataset_encoding_task.requires([tokenizer_training_task, dataset_loading_task])
train_task.requires([dataset_encoding_task, tokenizer_training_task])
model_selection_task.requires([train_task])
evaluate_task.requires([model_selection_task])

### 3. Creating the List of Task Spec Instances
We pack all these task specs in a list which gets passed to FluidML. `Flow` internally creates the task instances.

In [ ]:
# all tasks
tasks = [dataset_loading_task, tokenizer_training_task, dataset_encoding_task, train_task, model_selection_task, evaluate_task]

### 4. Setting all Meta Parameters

In [ ]:
# dynamically get the directory of this script
current_dir = os.path.abspath('')

# define the base directory where all our task results will be stored in a structured way using LocalFileStore
base_dir = os.path.join(current_dir, 'seq2seq_experiments')

# select the number of workers (processes used to execute tasks in parallel)
num_workers = 2

# set force to 'selected' if you want to force-execute the selected task (task_to_execute has to be set, e.g. task_to_execute='Training') 
#   regardless previously task results being available. 
# set force to 'all' if all tasks in the pipeline have to be force-executed.
# set force to None if already existing tasks are skipped so that results cen be loaded from the store.
force = None  # choices [selected, all, None]

# Only run the pipeline up to the selected task (including the task). All successor tasks defined in the pipeline will not be executed.
task_to_execute = None

# try to use cuda GPU's if available
use_cuda = True

# set a global seed for the entire pipeline
seed = 1234

### 5. Define and instantiate Resources to share across all Tasks

We mentioned already during the `Training` task that FluidML enables the user to conveniently share resources across all tasks instead of providing them explicitely to each task individually.  
The user achieves this by creating his own Resource dataclass, which inherits from our `Resource` interface. In this dataclass we define all resources, in our case the seed, and the cuda device, which we make available to all tasks through the `self.resource` attribute.

```python
@dataclass
class TaskResource(Resource):
    device: str
    seed: int
```

Further, we utilize a little helper function to distribute our available cuda devices euqally across the number of defined workers.  
E.g. let's assume we selected `num_workers = 4` and we have access to two GPU's, the below function would return the following balanced list of devices:

```python
print(devices)
-> ['cuda:0', 'cuda:1', 'cuda:0', 'cuda:1']
```

In [ ]:
def get_balanced_devices(count: Optional[int] = None,
                         use_cuda: bool = True) -> List[str]:
    count = count if count is not None else multiprocessing.cpu_count()
    if use_cuda and torch.cuda.is_available():
        devices = [f'cuda:{id_}' for id_ in range(torch.cuda.device_count())]
    else:
        devices = ['cpu']
    factor = int(count / len(devices))
    remainder = count % len(devices)
    devices = devices * factor + devices[:remainder]
    return devices

devices = get_balanced_devices(count=num_workers, use_cuda=use_cuda)

Finally, we create our list of resource objects which we will feed to the `Swarm` class during instantiation.  
**Note**: `len(resources) == num_workers`

In [ ]:
# create list of resources
resources = [TaskResource(device=devices[i], seed=seed) for i in range(num_workers)]

### 6. Instantiate the previously defined File Store

In [ ]:
# create local file storage used for versioning
results_store = MyLocalFileStore(base_dir=base_dir)

### 7. Run the Pipeline/Task-Graph

We use a context manager to instantiate the `Swarm` class, pass the swarm instance to the `Flow` class and finally run the pipeline.  
All saved task results will be availabe in the previously selected output directory (if not changed: `seq2seq_experiments`).  
Feel free to play around with different hyperparameter combinations to improve your model's translation performance.

In [ ]:
with Swarm(n_dolphins=num_workers,
           resources=resources,
           results_store=results_store
           refresh_every=1) as swarm:
    flow = Flow(swarm=swarm, task_to_execute=task_to_execute, force=force)
    flow.run(tasks)

<img src="https://raw.githubusercontent.com/fluidml/fluidml/main/logo/fluid_ml_logo.png" width=400 height=400 />